In [1]:
import tensorflow as tf 
import numpy as np
from tensorflow import keras

## Functional API 
Here we look at how the Multi input and output graph can be built. The problem set is that of series of support tickets with `title` `body` and `tags` added by the user to categorize the tickets. The data will have to be encoded as a series of 0 and 1 and this technique will be studied later and `vocabulary_size` will be the encoding space. 

The output of the model will be: 

1. priority of the ticket, scalar between 0 and 1 
2. department that should handle the ticket.

In [2]:
vocabulary_size = 10000
num_tags = 100
num_departments = 4 

In [6]:
titles = keras.Input(shape=(None,vocabulary_size), name='titles')
bodies = keras.Input(shape=(None,vocabulary_size), name='bodies')
tags = keras.Input(shape=(None,num_tags), name='tags')

features = keras.layers.Concatenate()([titles, bodies, tags])
features = keras.layers.Dense(64, activation='relu')(features)

priority = keras.layers.Dense(1, activation='sigmoid', name='priority')(features)
department = keras.layers.Dense(num_departments, name='department')(features)

ticket_model = keras.Model(inputs=[titles, bodies, tags], outputs=[priority, department])

In [15]:
# lets generate some data 
num_samples = 128 

# title data 
title_data = np.random.randint(0,2, size=(num_samples,vocabulary_size))
# text data 
text_data = np.random.randint(0,2, size=(num_samples,vocabulary_size))
# tages data 
tags_data = np.random.randint(0,2, size=(num_samples,num_tags))

priority_data = np.random.random(size=(num_samples, 1)).astype(np.float32)
department_data = np.random.randint(0,2, size=(num_samples, num_departments))

ticket_model.compile(optimizer='rmsprop', 
                     loss=['mean_squared_error', 'categorical_crossentropy'],
                     metrics=[['mean_absolute_error'], ['accuracy']])

ticket_model.fit([title_data, text_data, tags_data],
                [priority_data, department_data],
                epochs=1)

ticket_model.evaluate([title_data, text_data, tags_data], [priority_data, department_data])

priority, department = ticket_model.predict([title_data, text_data, tags_data])

print(f"Priority[0]: {priority[0]}")
print(f"Department[0]: {department[0]}")



4/4 [==============================] - 0s 3ms/step
Priority[0]: [8.697633e-09]
Department[0]: [ 37.828503 -41.801437 -28.662525  12.979788]


In [21]:
keras.utils.plot_model(ticket_model, 'multi_input_and_output_model.png')

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [22]:
ticket_model.layers